# <center> intention when writing </center>



In [1]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras import preprocessing
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import *
from tensorflow.keras.utils import *
import tensorflow as tf
from sklearn.decomposition import PCA
from matplotlib import cm, pyplot as plt

import numpy as np
old = np.load

np.load = lambda *a,**k: old(*a, **k)

In [2]:

max_features = 10000 # Número máximo de palabras diferentes de nuestro vocabulario.
maxlen = 300         # Número máximo de palabras en cada texto.

# Carga el texto ya tokenizado.
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features) 

# Preprocesado de los textos para igualar todas sus longitudes.
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test =  preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen) 

# Utiliza esta línea para obtener la palabra asociada a cada índice.
word_index = imdb.get_word_index()
index_word = {v: k for k, v in imdb.get_word_index().items()}

1646592/1641221 [==============================] - 1s 0us/step


In [3]:
print (x_test)

[[   0    0    0 ...   14    6  717]
 [   0    0    0 ...  125    4 3077]
 [1239 5189  137 ...    9   57  975]
 ...
 [   0    0    0 ...   21  846 5518]
 [   0    0    0 ... 2302    7  470]
 [   0    0    0 ...   34 2005 2643]]


In [4]:
x_freq = x_train[0:1000,:]

unique, counts = np.unique(x_freq, return_counts=True)
e = [unique, counts]
e = np.asarray(e)
e = e.T

e = e[e[:,1].argsort()]

sol = e[8510:, 0]


for i in range(50):
    print(index_word.get(sol[i]))

there
some
very
good
about
has
out
if
it's
what
like
so
from
or
who
her
just
they
all
the
an
by
be
he
one
at
his
are
have
you
film
on
but
not
movie
with
for
as
was
that
this
i
it
in
br
is
to
and
of
None


In [9]:
model = Sequential()

model.add(Embedding(max_features,32, input_length= maxlen))
model.add(tf.keras.layers.LSTM(32, return_sequences=True))
model.add(Flatten())
model.add(Dense(30,activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy',  metrics=['accuracy'])

model.fit(x_train,y_train,epochs=5, validation_data=(x_test,y_test))

Epoch 1/10
782/782 [==============================] - 16s 21ms/step - loss: 0.3456 - accuracy: 0.8423 - val_loss: 0.2898 - val_accuracy: 0.8773
Epoch 2/10
782/782 [==============================] - 16s 21ms/step - loss: 0.1941 - accuracy: 0.9244 - val_loss: 0.3119 - val_accuracy: 0.8749
Epoch 3/10
782/782 [==============================] - 16s 21ms/step - loss: 0.1130 - accuracy: 0.9592 - val_loss: 0.3441 - val_accuracy: 0.8669
Epoch 4/10
782/782 [==============================] - 16s 21ms/step - loss: 0.0481 - accuracy: 0.9826 - val_loss: 0.5052 - val_accuracy: 0.8592
Epoch 5/10
782/782 [==============================] - 16s 21ms/step - loss: 0.0264 - accuracy: 0.9902 - val_loss: 0.6209 - val_accuracy: 0.8587
Epoch 6/10
782/782 [==============================] - 16s 21ms/step - loss: 0.0126 - accuracy: 0.9957 - val_loss: 0.7946 - val_accuracy: 0.8592
Epoch 7/10
782/782 [==============================] - 16s 21ms/step - loss: 0.0101 - accuracy: 0.9966 - val_loss: 0.8417 - val_accuracy:

In [10]:
x_train[0:1,:].shape

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 300, 32)           320000    
_________________________________________________________________
lstm (LSTM)                  (None, 300, 32)           8320      
_________________________________________________________________
flatten (Flatten)            (None, 9600)              0         
_________________________________________________________________
dense (Dense)                (None, 30)                288030    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 616,381
Trainable params: 616,381
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
inp = Input(shape=(300,))

mod = Model(inputs=inp, outputs=model.layers[0](inp))

mod.predict(x_train[0:1,:])

array([[[ 0.01422531, -0.04462078,  0.01487336, ..., -0.0352954 ,
          0.04165985, -0.04081545],
        [ 0.01422531, -0.04462078,  0.01487336, ..., -0.0352954 ,
          0.04165985, -0.04081545],
        [ 0.01422531, -0.04462078,  0.01487336, ..., -0.0352954 ,
          0.04165985, -0.04081545],
        ...,
        [ 0.03756596,  0.00245842,  0.04154585, ...,  0.06430374,
         -0.02673352, -0.01992405],
        [ 0.09998078, -0.06040833, -0.10473489, ...,  0.12870932,
         -0.05075734,  0.05886845],
        [-0.06570345,  0.04742532,  0.01181766, ..., -0.04680649,
          0.10554241, -0.01379974]]], dtype=float32)

In [12]:
data = x_train[2:3,:]

pre = model.predict_classes(data)
print (pre)

data1 = np.ravel(data)

for i in range(data.size):
    if (data1[i] != 0):
        print (index_word.get(data1[i]))


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[[0]]
the
as
there
in
at
by
br
of
sure
many
br
of
proving
no
only
women
was
than
doesn't
as
you
never
of
hat
night
that
with
ignored
they
bad
out
superman
plays
of
how
star
so
stories
film
comes
defense
date
of
wide
they
don't
do
that
had
with
of
hollywood
br
of
my
seeing
fan
this
of
pop
out
body
shots
in
having
because
cause
it's
stick
passing
first
were
enjoys
for
from
look
seven
sense
from
me
and
die
in
character
as
cuban
issues
but
is
you
that
isn't
one
song
just
is
him
less
are
strongly
not
are
you
that
different
just
even
by
this
of
you
there
is
eight
when
it
part
are
film's
love
film's
80's
was
big
also
light
don't
and
as
it
in
character
looked
cinema

In [13]:
data = "In markets such as the UK, where high-priced contracts are commonplace, such a low handset-only pricing should prove enticing to those after a high-end experience without the income to match. The only question mark falls to the phone’s long-term support, with an unconfirmed path for OS and security updates going forward.Elsewhere, the hardware and software experience feels as tight as any top-tier phone you’d find on store shelves right now – which get you thinking about what the higher price tag of phones such as the iPhone XS actually affords you."

data = data.split(' ')
data = np.asarray(data)

e = np.zeros((1,300))

for i in range(data.size):
    if (word_index.get(data[i]) != None):    
        e[0,299 - i] = word_index.get(data[i])


        

In [14]:

pre = model.predict_classes(x_train[4:5,:])

print (pre)

[[0]]


In [15]:
from sklearn.decomposition import PCA
from matplotlib import cm, pyplot as plt

In [ ]:
X = mod.predict(x_train)
plt.scatter(X[:,0], X[:,1])
plt.show()
pca = PCA()
X = pca.fit(X)
plt.scatter(X[:,0], X[:,1])
plt.show()
